
# CrewAI with AgentNeo Integration
This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with CrewAI, a framework for orchestrating role-playing AI agents. This integration allows for comprehensive analysis and debugging of AI-powered systems.

# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [1]:
import os
os.chdir('../..')

In [2]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv

from agentneo import AgentNeo, Tracer ,Evaluation, launch_dashboard

# Load environment variables
load_dotenv("enter your .env path")

False


# Initialize AgentNeo Session and Tracer
Now, let's set up our AgentNeo session and tracer.

In [3]:
# Initialize AgentNeo Session and Tracer
neo_session = AgentNeo(session_name="marketing_campaign_example103")

# Connect to a Project
neo_session.create_project(project_name="marketing_campaign_project103")

# Create tracer
tracer = Tracer(session=neo_session)

tracer.start()


ValueError: Project 'marketing_campaign_project103' already exists.


# Define Helper Functions
Let's define some helper functions for our AI tools, using AgentNeo's tracing capabilities.

In [5]:
from langchain_community.tools import TavilySearchResults

seo_tool = tracer.wrap_langchain_tool(TavilySearchResults())

# Define Agents
Now, let's create our AI agents using CrewAI.

In [6]:
# Define Agents
@tracer.trace_agent("strategist")
def create_strategist():
    return Agent(
        role='Marketing Strategist',
        goal='Develop a high-impact marketing campaign for a new AI product',
        backstory="""You are a seasoned marketing strategist at a leading tech company.
        You excel at creating data-driven campaigns that resonate with target audiences.""",
        verbose=True,
        allow_delegation=False,
        tools=[seo_tool],
    )

@tracer.trace_agent("designer")
def create_designer():
    return Agent(
        role='Creative Director',
        goal='Design visually appealing assets for a marketing campaign',
        backstory="""You are a Creative Director with a flair for developing innovative designs.
        You translate marketing concepts into visual masterpieces.""",
        verbose=True,
        allow_delegation=True,
    )

strategist = create_strategist()
designer = create_designer()


DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 16
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 17


# Define Tasks
Let's create tasks for our agents.

In [7]:
# Define Tasks
task1 = Task(
  description="""Develop a comprehensive marketing strategy for launching a new AI product.
  Consider SEO, target demographics, and competition analysis.""",
  expected_output="Full marketing strategy document",
  agent=strategist
)

task2 = Task(
  description="""Create design assets (e.g., social media banners, infographics) based on the marketing strategy.
  Ensure consistency with the product's brand identity.""",
  expected_output="Designs in PNG format",
  agent=designer
)



# Create and Execute Crew
Now, let's create our crew and execute the tasks.

In [1]:
# Create and Execute Crew
crew = Crew(
  agents=[strategist, designer],
  tasks=[task1, task2],
  process=Process.sequential,
  verbose=True,
)

# Get your crew to work!
result = crew.kickoff()

print(result)

tracer.stop()

NameError: name 'Crew' is not defined

# Metrics Evaluation

In [8]:
from agentneo import Evaluation
exe = Evaluation(session=neo_session, trace_id=1)

# run a single metric
exe.evaluate(metric_list=['tool_call_success_rate'])


In [ ]:
#print metric result
metric_results = exe.get_results()
print(metric_results)

In [ ]:
neo_session.launch_dashboard(port=3000)